# Company name matching

See stackoverflow page:
http://stackoverflow.com/questions/39680624/matching-company-names-in-the-news-data-using-python


In [1]:
# Imports

import sys, itertools, csv, nltk, re, editdistance

from pyexcel_xlsx import get_data

import pandas as pd


# Import spacy and English models
import spacy, numpy

# Load English Spacy module
nlp = spacy.load('en')

In [8]:
# Import data - using pyexcel

datain1 = get_data("ftse100companies.xlsx")
datain2 = get_data("companytestlist.xlsx")

datain2

OrderedDict([('Sheet1',
              [['Company'],
               ['3i'],
               ['Aberdeen Asset Management'],
               ['Admiral Group'],
               ['British Petroleum'],
               ['BAT'],
               ['BAE'],
               ['Barratt'],
               ['Direct Line'],
               ['LSE'],
               ['RBS'],
               ['BT'],
               ['CocaCola'],
               ['Jonson Mathey'],
               ['Sports Drect'],
               ['Tescos'],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
               [],
           

In [39]:
# Import data - using pandas

df1 = pd.read_excel("ftse100companies.xlsx")
df2 = pd.read_excel("companytestlist.xlsx")

df1

,Company
0,3i
1,Aberdeen Asset Management
2,Admiral Group
3,Anglo American plc
4,Antofagasta
5,ARM Holdings
6,Ashtead Group
7,Associated British Foods
8,AstraZeneca
9,Aviva


In [9]:
# Get company name lists from imported excel data

items1 = list(itertools.chain.from_iterable(datain1.items()))
items1trimmed = items1[1][1:]

items2 = list(itertools.chain.from_iterable(datain2.items()))
items2trimmed = items2[1][1:]


# Exact matching

In [10]:
# Exact match (using pyexcel formatted input)

matched=[]

for item in items1trimmed:
    if item in items2trimmed:
        print(item)
        matched.append(item)


['3i']
['Aberdeen Asset Management']
['Admiral Group']


In [70]:
# Output matches

with open("company_name_matches.csv", 'w', newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerows(matched)

# Fuzzy matching

Four cases to deal with:

1. There can be variations like LTD, Ltd, Limited, Corp, Corporation. & and etc
2. There can be abbreviations like IBM and International Business Machine
3. There can be typographical errors, deliberately as part of company branding (e.g. flickr / flicker) or accidentally (e.g. Orcle / Oracle)
4. There can be omissions - Goyal and Sons, Goyal

In [3]:
# Function to strip known trailing company type extensions - lower case for now

co_extensions = ['limited', 'ltd', 'ltd.', 'corp', 'corp.', 'inc', 'inc.', 'plc', 'Group']
co_extensions = [item.upper() for item in co_extensions]
co_extensions

['LIMITED', 'LTD', 'LTD.', 'CORP', 'CORP.', 'INC', 'INC.', 'PLC', 'GROUP']

In [14]:
# Function to strip ending 

def strip_end(text, suffix):
    if not text.endswith(suffix):
        return text
    else:
        return text[:len(text)-len(suffix)]

In [15]:
# Function to remove known company extensions (deals with cases 1 and 4)

def strip_extensions(data_in, co_extensions):

    shortened=[]

    for item in data_in:
        for subitem in item:
            temp_string = subitem.upper()
            for ext in co_extensions:
                temp_string = strip_end(temp_string, ext)
            shortened.append(temp_string.strip())

    return shortened

In [32]:
# Call remove extensions function - pyxexcel formatted data

shortened1 = strip_extensions(items1trimmed, co_extensions)
shortened2 = strip_extensions(items2trimmed, co_extensions)
shortened2

['3I',
 'ABERDEEN ASSET MANAGEMENT',
 'ADMIRAL',
 'BRITISH PETROLEUM',
 'BAT',
 'BAE',
 'BARRATT',
 'DIRECT LINE',
 'LSE',
 'RBS',
 'BT',
 'COCACOLA',
 'JONSON MATHEY',
 'SPORTS DRECT',
 'TESCOS']

In [35]:
# Function to strip extensions, using LIST as input

def strip_extension_list(data_in, co_extensions):

    shortened=[]

    for item in data_in:
        temp_string = item.upper()
        for ext in co_extensions:
            temp_string = strip_end(temp_string, ext)
        shortened.append(temp_string.strip())

    return shortened

In [42]:
# Call remove extensions function - dataframe format


# Convert Company name to list
dflist1 = df1['Company'].tolist()
dflist2 = df2['Company'].tolist()


# Call remove extensions function - pyxexcel formatted data

dflistshortened1 = strip_extension_list(dflist1, co_extensions)
dflistshortened2 = strip_extension_list(dflist2, co_extensions)

# Put back into DataFrame
df1['shortened'] = dflistshortened1
df2['shortened'] = dflistshortened2

df2

,Company,shortened
0,3i,3I
1,Aberdeen Asset Management,ABERDEEN ASSET MANAGEMENT
2,Admiral Group,ADMIRAL
3,British Petroleum,BRITISH PETROLEUM
4,BAT,BAT
5,BAE,BAE
6,Barratt,BARRATT
7,Direct Line,DIRECT LINE
8,LSE,LSE
9,RBS,RBS


In [70]:
# Flag if abbreviation variant is relevant - PYEXCEL version
# Only relevant if there are more than one term
def abbreviate(data_in):
    
    abbreviations=[]
    
    for item in data_in:
        subitem_split = item.split()
        initials = ""
        if len(subitem_split) > 1:
            for word in subitem_split:
                initials += word[0]
            abbreviations.append(initials)
    
    return abbreviations


# Call abbreviations function

abbreviations1 = abbreviate(shortened1)
abbreviations2= abbreviate(shortened2)
abbreviations2

,Company,shortened,word_count,split,acronym
0,3i,3I,1,[3I],3i
1,Aberdeen Asset Management,ABERDEEN ASSET MANAGEMENT,3,"[ABERDEEN, ASSET, MANAGEMENT]",Aberdeen Asset Management
2,Admiral Group,ADMIRAL,1,[ADMIRAL],Admiral Group
3,Anglo American plc,ANGLO AMERICAN,2,"[ANGLO, AMERICAN]",Anglo American plc
4,Antofagasta,ANTOFAGASTA,1,[ANTOFAGASTA],Antofagasta
5,ARM Holdings,ARM HOLDINGS,2,"[ARM, HOLDINGS]",ARM Holdings
6,Ashtead Group,ASHTEAD,1,[ASHTEAD],Ashtead Group
7,Associated British Foods,ASSOCIATED BRITISH FOODS,3,"[ASSOCIATED, BRITISH, FOODS]",Associated British Foods
8,AstraZeneca,ASTRAZENECA,1,[ASTRAZENECA],AstraZeneca
9,Aviva,AVIVA,1,[AVIVA],Aviva


In [79]:
# Flag if abbreviations variant is relevant

# Convert to list
dflistshortened1 = df1['shortened'].tolist()
dflistshortened2 = df2['shortened'].tolist()


# Function to abbreviate from LIST input
def abbreviate(data_in):
    
    abbreviations=[]
    
    for item in data_in:
        subitem_split = item.split()
        initials = ""
        if len(subitem_split) > 1:
            for word in subitem_split:
                initials += word[0]
        abbreviations.append(initials)
    
    return abbreviations


# Call abbreviations function
abbreviations1 = abbreviate(dflistshortened1)
abbreviations2= abbreviate(dflistshortened2)


# Put back into DataFrame
df1['acronym'] = abbreviations1
df2['acronym'] = abbreviations2

df2


,Company,shortened,acronym
0,3i,3I,
1,Aberdeen Asset Management,ABERDEEN ASSET MANAGEMENT,AAM
2,Admiral Group,ADMIRAL,
3,British Petroleum,BRITISH PETROLEUM,BP
4,BAT,BAT,
5,BAE,BAE,
6,Barratt,BARRATT,
7,Direct Line,DIRECT LINE,DL
8,LSE,LSE,
9,RBS,RBS,


In [107]:
# Edit Distance
def closesteditdistancematch(targetlist, referencelist):
    
    returnlist = []
    
    for targetitem in targetlist:
        closestmatch=""
        closestdistance=999
        for referenceitem in referencelist:
            currenteditdistance = editdistance.eval(targetitem, referenceitem)
            if  currenteditdistance < closestdistance:
                closestmatch = referenceitem
                closestdistance = currenteditdistance
        returnlist.append([closestmatch, closestdistance])       

    return returnlist

editdistance_21 = closesteditdistancematch(dflistshortened2, dflistshortened1)

editdistarray = numpy.asarray(editdistance_21)

# len(editdistarray[:, 0])

df2['editdistmatch'] = editdistarray[:, 0]
df2['editdistance'] = editdistarray[:, 1]

df2

# Scrap code

,Company,shortened,acronym,editdistmatch,editdistance
0,3i,3I,,3I,0
1,Aberdeen Asset Management,ABERDEEN ASSET MANAGEMENT,AAM,ABERDEEN ASSET MANAGEMENT,0
2,Admiral Group,ADMIRAL,,ADMIRAL,0
3,British Petroleum,BRITISH PETROLEUM,BP,BRITISH LAND,8
4,BAT,BAT,,BT,1
5,BAE,BAE,,BP,2
6,Barratt,BARRATT,,BARCLAYS,4
7,Direct Line,DIRECT LINE,DL,DIRECT LINE,0
8,LSE,LSE,,SSE,1
9,RBS,RBS,,BP,2


In [112]:
# Merge on acronyms

df_merged_acronyms = pd.merge(df2, df1[df1['acronym'] != ''], left_on='acronym', right_on='acronym', how='left')

df_merged_acronyms

,Company_x,shortened_x,acronym,editdistmatch,editdistance,Company_y,shortened_y,word_count,split
0,3i,3I,,3I,0,NaN,NaN,NaN,NaN
1,Aberdeen Asset Management,ABERDEEN ASSET MANAGEMENT,AAM,ABERDEEN ASSET MANAGEMENT,0,Aberdeen Asset Management,ABERDEEN ASSET MANAGEMENT,3.0,"[ABERDEEN, ASSET, MANAGEMENT]"
2,Admiral Group,ADMIRAL,,ADMIRAL,0,NaN,NaN,NaN,NaN
3,British Petroleum,BRITISH PETROLEUM,BP,BRITISH LAND,8,NaN,NaN,NaN,NaN
4,BAT,BAT,,BT,1,NaN,NaN,NaN,NaN
5,BAE,BAE,,BP,2,NaN,NaN,NaN,NaN
6,Barratt,BARRATT,,BARCLAYS,4,NaN,NaN,NaN,NaN
7,Direct Line,DIRECT LINE,DL,DIRECT LINE,0,Direct Line Group,DIRECT LINE,2.0,"[DIRECT, LINE]"
8,LSE,LSE,,SSE,1,NaN,NaN,NaN,NaN
9,RBS,RBS,,BP,2,NaN,NaN,NaN,NaN
